# 1. Installing Dependencies

!pip install -q datasets transformers accelerate timm
!pip install -q -U albumentations>=1.4.5 torchmetrics pycocotools

In [ ]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim import Adam, SGD
import torch.nn.functional as F

import torchvision
from torchvision.transforms import transforms
from torchvision.models import models

# Building and Playing with DATA!

from roboflow import Roboflow
rf = Roboflow(api_key="here")
project = rf.workspace("roboflow-100").project("uno-deck")
version = project.version(2)
dataset = version.download("yolov8")

In [ ]:
img_path = '/content/uno.jpg'


model_1 = YOLO('yolov8n.pt')
model_2 = YOLO('yolov8n.pt')
model_3 = YOLO('yolov8n.pt')
model_4 = YOLO('yolov8n.pt')


img = cv2.imread(img_path) 

result_1 = model_1(img, stream = True) 
result_2 = model_2(img, stream = True) 
result_3 = model_3(img, stream = True) 
result_4 = model_4(img, stream = True)

In [ ]:
from PIL import Image, ImageDraw, ImageFont

imgt = Image.open('/content/uno.jpg')
draw = ImageDraw.Draw(imgt)
class_names = model_1.names

for result in result_1:
  boxes = result.boxes.xyxy.cpu().numpy()
  classes = result.boxes.cls.cpu().numpy()
  score = result.boxes.conf.cpu().numpy()

  for box, cls, sc in zip(boxes, classes, score):
    x1,y1,x2,y2 = [int(v) for v in box]
    draw.rectangle([x1,y1,x2,y2], outline = 'red', width = 3)
    label = f"{class_names[cls]}({sc:.2f})"
    draw.text((x1,y1-10), label, fill = 'red')

display(imgt)

In [ ]:
def draw_predictions(image, results, model_name):
    
    img_copy = image.copy()
    draw = ImageDraw.Draw(img_copy)
    class_names = results[0].names
    
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy()
        scores = result.boxes.conf.cpu().numpy()
        
        for box, cls, score in zip(boxes, classes, scores):
            x1, y1, x2, y2 = [int(v) for v in box]
            draw.rectangle([x1, y1, x2, y2], outline='red', width=3)
            label = f"{class_names[int(cls)]}({score:.2f})"
            draw.text((x1, y1-10), label, fill='red')
    
    return img_copy


for (model_name, model), ax in zip(models.items(), axs.flat):
    # Get predictions
    results = model(img)
    
    # Draw predictions on image
    predicted_img = draw_predictions(img, results, model_name)
    
    # Plot on subplot
    ax.imshow(predicted_img)
    ax.set_title(f'YOLOv8 {model_name.capitalize()}')
    ax.axis('off')


In [ ]:
def compare_detection_metrics(models, img):
    metrics = {}
    for name, model in models.items():
        results = model(img)
        result = results[0]
        
        metrics[name] = {
            'total_objects': len(result.boxes),
            'avg_confidence': result.boxes.conf.cpu().numpy().mean(),
            'max_confidence': result.boxes.conf.cpu().numpy().max(),
            'unique_classes': len(np.unique(result.boxes.cls.cpu().numpy()))
        }
    
    # Create comparative bar plot
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    metrics_to_plot = ['total_objects', 'avg_confidence', 'max_confidence', 'unique_classes']
    

In [ ]:
def inference_speed_comparison(models, img):
    import time
    
    speed_results = {}
    for name, model in models.items():
        start_time = time.time()
        model(img)
        end_time = time.time()
        speed_results[name] = end_time - start_time
    # inf models
    plt.figure(figsize=(10, 6))
    plt.bar(speed_results.keys(), speed_results.values())
    plt.title('Inference Time Comparison')
    plt.ylabel('Time (seconds)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show() 